In [ ]:
!pip install pyzbar
!apt-get install libzbar0

import cv2
import numpy as np
from pyzbar.pyzbar import decode
from google.colab.patches import cv2_imshow

def detect_barcode_from_webcam():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not access the webcam.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Webcam FPS: {fps} frames per second")

    while True:
        ret, image = cap.read()

        if not ret:
            print("Error: Failed to capture image from webcam.")
            break

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 240)
        cv2.imshow("Edges", edges)

        orb = cv2.ORB_create(nfeatures=500)
        keypoints, descriptors = orb.detectAndCompute(edges, None)
        keypoint_image = cv2.drawKeypoints(image, keypoints, None, color=(0, 255, 0), flags=0)
        decoded_objects = decode(keypoint_image)

        if decoded_objects:
            for obj in decoded_objects:
                barcode_type = obj.type
                barcode_data = obj.data.decode("utf-8")
                print(f"Type: {barcode_type}, Data: {barcode_data}")
                cv2.putText(image, f"Barcode: {barcode_data}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            break

        cv2_imshow("Barcode Detection", image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

detect_barcode_from_webcam()